# Les sorties ciné de 2023

## Objectif : 
Obtenir à l'aide de la technique de webscraping de récupérer une liste de films qui sortirons en 2023 à exporter en CSV pour les mettres ensuite dans Notion pour le suivi des vus/on vus

**Source** : 
- https://www.premiere.fr/Cinema/News-Cinema/Les-80-films-les-plus-attendus-de-2023-au-cinema-et-en-streaming--preview


### Etape 1 : importer les libriaires nécessaire pour la webscraping

In [4]:
#Importation des librairies nécessaires pour le script
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
#import scrapy
import re
import requests

import json
from pandas.io.json import json_normalize

import pandas as pd
import numpy as np
import datetime as dt  #pour l'ajout de la date de l'extraction de données
from datetime import datetime
import os

#from pandas.io.json import json_normalize #uniquement si ce n'est pas la dernière version de pandas



### Etape 2 : webscraping avec BeautifulSoup

### liste films Première

In [5]:
# url à scraper
url = 'https://www.premiere.fr/Cinema/News-Cinema/Les-80-films-les-plus-attendus-de-2023-au-cinema-et-en-streaming--preview'

In [6]:
#your code
trend = requests.get(url).content
trend_soup = BeautifulSoup(trend,"lxml")
data_film=trend_soup.findAll('p',attrs={"dir":"ltr"})
data_film

[<p dir="ltr"><strong><a href="https://www.premiere.fr/film/Terrifier-2"><em>Terrifier 2</em></a> - le 11 janvier</strong></p>,
 <p dir="ltr">Un clown psychopathe et un jusqu’au-boutisme gore affolant : pas besoin d’avoir vu le premier <em>Terrifier</em> pour s’intéresser à cette suite en forme de farce sanguinolente (succès surprise aux États-Unis), qui paye son hommage aux <em>Griffes de la nuit</em> et à <em>Evil Dead 2</em>.</p>,
 <p dir="ltr"><strong><a href="https://www.premiere.fr/film/Babylon-3"><em>Babylon</em></a> de Damien Chazelle - le 18 janvier</strong></p>,
 <p dir="ltr">La naissance de Hollywood racontée par le réalisateur de <em>La La Land</em> ? Trois heures de fêtes, d'orgies et de démesure, rythmées par le jazz fou de Justin Hurwitz ? <strong><em>Babylon</em></strong> sera sans aucun doute l'un des films les plus fous de l'année 2023 !</p>,
 <p dir="ltr"><strong><a href="https://www.premiere.fr/film/Tar-0"><em>Tàr</em></a> de Todd Field – le 25 janvier</strong></p>,

In [7]:
dflib = pd.DataFrame([item.text for d in data_film for item in d.findAll('strong')], columns={'film'})


In [8]:
dflib = pd.DataFrame([item.text for d in data_film for item in d.findAll('strong')])
pattern_date = r' - | – |- | : '
pattern_plateforme = r' .sur | sur+'
dflib = dflib[0].str.split(pattern_date, expand=True).rename(columns={0:'film',1:'col1',2:'col2',3:'col3'})
dflib['col1'] = dflib['col1'].str.split(pattern_plateforme)

#Je corrige les erreurs de split des colonnes 2 & 3

dflib.film[5] = "Astérix et Obélix - L'empire du milieu de Guillaume Canet"
dflib.col1[5] = 'le 1er février'

dflib.film[17] = '65 - La Terre d’avant de Scott Beck et Bryan Woods'
dflib.col1[17] = 'le 15 mars'

dflib.film[27] = 'Les Trois mousquetaires - d’Artagnan et Les Trois mousquetaires - Milady de Martin Bourboulon'
dflib.col1[27] = 'les 25 avril et 13 décembre'

dflib.film[38] = 'Mission : Impossible - Dead Reckoning partie 1'
dflib.col1[38] = 'le 12 juillet'

dflib.film[53] = 'L’Abbé Pierre - Une vie de combats de Frédéric Tellier'
dflib.col1[53] = 'le 9 novembre'

#Je supprime ensuite les colonnes 2 & 3
dflib = dflib.drop(columns=["col2", "col3"])

#J'identifie et supprime les lignes inutiles car je dois avoir que 80 films et non 86
dflib.loc[dflib.col1.isnull()]

,film,col1
2,Babylon,None
19,John Wick 4,None
44,Gran Turismo,None
52,,None
54,,None
57,,None
60,,None


In [9]:
dflib = dflib.drop([2,19,50,52,55,58])
dflib.reset_index(inplace = False)
dflib.rename(columns={'col1':'date'})

,film,date
0,Terrifier 2,[le 11 janvier]
1,Babylon de Damien Chazelle,[le 18 janvier]
3,Tàr de Todd Field,[le 25 janvier]
4,You People de Kenya Barris,"[le 27 janvier, Netflix ]"
5,Astérix et Obélix - L'empire du milieu de Guil...,le 1er février
...,...,...
83,Napoléon de Ridley Scott,"[sans date, Apple TV+) ]"
84,La Tortue de Léa Domenach,[sans date]
85,Challengers de Luca Guadagnino,[sans date ]
86,The Nightingale de Mélanie Laurent,[sans date]


### Films from Sens critique

In [3]:
hdr = {'User-Agent': 'Mozilla/5.0'}
titre=[]
lien=[]
sortie=[]
creator=[]

for x in range(3):
    site= "https://www.senscritique.com/liste/films_les_plus_attendus_en_2023/2994033?page="+str(x+1)
    req = Request(site,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)
    film_soup = soup.find_all("div", attrs={'class':'ProductListItem__Content-sc-ico7lo-5 dHdDpi'})
    
    try:
        for film in film_soup :
            #print(film.find('a').text)
            titre.append(film.find('a').text)
            #print("https://www.senscritique.com"+film.find('a').get("href"))
            lien.append("https://www.senscritique.com"+film.find('a').get("href"))
            #print(film.find('span',attrs={'data-testid':"date-release"}).text)
            sortie.append(film.find('span',attrs={'data-testid':"date-release"}).text)
            if str(film.find('span',href=True).text) == '':
                #print("Creator : NC")
                creator.append("NC")
            else :
                #print("Creator : "+film.find('span',href=True).text)
                creator.append(film.find('span',href=True).text)
            
    except Exception as e:
        #print(film.find('span',attrs={'class':"Text__SCUserText-sc-14ie3lm-2 kQEuxe"}).text)
        sortie.append(film.find('span',attrs={'class':"Text__SCUserText-sc-14ie3lm-2 kQEuxe"}).text)
        #print(film.find('a',attrs={'Text__SCText-sc-14ie3lm-0 Link__PrimaryLink-sc-1vfcbn2-0 fzhdFi dPapvk'}).text)
        creator.append(film.find('a',attrs={'Text__SCText-sc-14ie3lm-0 Link__PrimaryLink-sc-1vfcbn2-0 fzhdFi dPapvk'}).text)

In [4]:
liste_senscritique = pd.DataFrame(list(zip(titre, lien, sortie, creator)),columns =['titre', 'lien', 'sortie', 'creator'])

liste_senscritique["titre"]= liste_senscritique["titre"].str.split("(")
liste_senscritique["Titre"]=[x[0] for x in liste_senscritique["titre"]]
liste_senscritique = liste_senscritique.drop("titre", axis=1)
liste_senscritique = liste_senscritique[['Titre','lien','sortie','creator']]
liste_senscritique

,Titre,lien,sortie,creator
0,Tirailleurs,https://www.senscritique.com/film/tirailleurs/...,Sortie : 4 janvier 2023.,Mathieu Vadepied
1,The Pale Blue Eye,https://www.senscritique.com/film/the_pale_blu...,Sortie : 6 janvier 2023 (France).,Scott Cooper
2,Terrifier 2,https://www.senscritique.com/film/terrifier_2/...,Sortie : 11 janvier 2023 (France).,Damien Leone
3,Babylon,https://www.senscritique.com/film/babylon/3975...,Sortie : 18 janvier 2023 (France).,Damien Chazelle
4,Devotion,https://www.senscritique.com/film/devotion/475...,Sortie : 20 janvier 2023 (France).,J.D. Dillard
...,...,...,...,...
61,Les Trolls 3,https://www.senscritique.com/film/les_trolls_3...,Sortie : 18 octobre 2023 (États-Unis).,Tim Heitz
62,Les Trois Mousquetaires - Milady,https://www.senscritique.com/film/les_trois_mo...,Sortie : 13 décembre 2023.,Martin Bourboulon
63,Wonka,https://www.senscritique.com/film/wonka/35024078,Sortie : 13 décembre 2023 (France).,Paul King
64,Aquaman et le Royaume perdu,https://www.senscritique.com/film/aquaman_and_...,Sortie : 20 décembre 2023 (France).,James Wan


In [5]:
for link in liste_senscritique["lien"]:
    print(link+'/details')

https://www.senscritique.com/film/tirailleurs/45177319/details
https://www.senscritique.com/film/the_pale_blue_eye/44258720/details
https://www.senscritique.com/film/terrifier_2/40741730/details
https://www.senscritique.com/film/babylon/39750335/details
https://www.senscritique.com/film/devotion/47565058/details
https://www.senscritique.com/film/vaincre_ou_mourir/48429393/details
https://www.senscritique.com/film/mayday/48629795/details
https://www.senscritique.com/film/tar/45408140/details
https://www.senscritique.com/film/teen_wolf_le_film/45446763/details
https://www.senscritique.com/film/asterix_obelix_l_empire_du_milieu/24411806/details
https://www.senscritique.com/film/knock_at_the_cabin/45450929/details
https://www.senscritique.com/film/le_pire_voisin_du_monde/44043788/details
https://www.senscritique.com/film/alibi_com_2/46531865/details
https://www.senscritique.com/film/magic_mike_s_last_dance/45499008/details
https://www.senscritique.com/film/winnie_the_pooh_blood_and_honey/4

### IMDB

In [277]:
#Je récupère les titres qui vont sortir dans l'année. 
#MAIS cette liste en fonction de la semaine où on récupère, on perd les info des semaines passées

#Créaton des colonnes de base depuis IMDB
imdb_id_film=[]
imdb_titre=[]

#Création de la colonne des infos de film depuis l'API TMDB à l'aide de l'ID IMDB 
imdb_info=[] #liste titres non trouvés sur MovieDB par le biais de l'api


#création de colonnes depuis imdb_info
adult = []
backdrop_path = []
id_ = []
title = []
original_language = []
original_title = []
overview = []
poster_path = []
media_type = []
genre_ids = []
popularity = []
release_date = []
video = []
vote_average = []
vote_count = []

#Je cherche la liste de filme et récupère l'ID et le film
site_imdb= "https://www.imdb.com/calendar/?ref_=nv_mv_cal&rlm&region=fr&type=MOVIE"
req_imdb = Request(site_imdb,headers=hdr)
page_imdb = urlopen(req_imdb)
soup_imdb = BeautifulSoup(page_imdb)
film_soup_imdb = soup_imdb.find_all('article',attrs={'data-testid':"calendar-section"})

for f in film_soup_imdb:
    film_info = f.findAll('li', attrs={'data-testid':'coming-soon-entry'})
    for film in film_info :
        imdb_id = film.find('a').get('href').replace('/title/','').replace('/?ref_=rlm','')
        imdb_id_film.append(imdb_id)
        imdb_titre.append(film.find('a').text)
        #Ici je récupère les infos du film avec l'ID IMDB sur l'API TMDB
        url_mdb = 'https://api.themoviedb.org/3/find/'+imdb_id+'?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=en-US&external_source=imdb_id'
        response_l = requests.get(url_mdb)
        json_data_l = response_l.json()
        df_n=pd.json_normalize(json_data_l['movie_results'])
        imdb_info.append(df_n)
        
        #ajout des info dans le dataframe df_n
        if len(df_n)==1 :
            adult.append(df_n['adult'][0])
            backdrop_path.append(df_n['backdrop_path'][0])
            id_.append(df_n['id'][0])
            title.append(df_n['title'][0])
            original_language.append(df_n['original_language'][0])
            original_title.append(df_n['original_title'][0])
            overview.append(df_n['overview'][0])
            poster_path.append(df_n['poster_path'][0])
            media_type.append(df_n['media_type'][0])
            genre_ids.append(df_n['genre_ids'][0])
            popularity.append(df_n['popularity'][0])
            release_date.append(df_n['release_date'][0])
            video.append(df_n['video'][0])
            vote_average.append(df_n['vote_average'][0])
            vote_count.append(df_n['vote_count'][0])
        else:
            adult.append('')
            backdrop_path.append('')
            id_.append('')
            title.append('')
            original_language.append('')
            original_title.append('')
            overview.append('')
            poster_path.append('')
            media_type.append('')
            genre_ids.append('')
            popularity.append('')
            release_date.append('')
            video.append('')
            vote_average.append('')
            vote_count.append('')

In [278]:
#Création du dataframe à partir des colonnes créées
liste_imdb = pd.DataFrame(list(zip(imdb_id_film, imdb_titre, imdb_info,adult,backdrop_path,id_,title,original_language,original_title,overview,poster_path,media_type,genre_ids,popularity,release_date,video,vote_average,vote_count)),columns =['id', 'titre', 'info','adult',
'backdrop_path',
'id_',
'title',
'original_language',
'original_title',
'overview',
'poster_path',
'media_type',
'genre_ids',
'popularity',
'release_date',
'video',
'vote_average',
'vote_count'])
liste_imdb

,id,titre,info,adult,backdrop_path,id_,title,original_language,original_title,overview,poster_path,media_type,genre_ids,popularity,release_date,video,vote_average,vote_count
0,tt10640346,Babylon (2022),adult backdrop_path ...,False,/oCKZAdUROqdlTcUOstqJ1gM8JQt.jpg,615777,Babylon,en,Babylon,A tale of outsized ambition and outrageous exc...,/wjOHjWCUE0YzDiEzKv8AfqHj3ir.jpg,movie,"[18, 35]",150.735,2022-12-23,False,7.46,50
1,tt11930126,Nos soleils (2022),adult backdrop_path ...,False,/5sl6GEHjvo0V2lGYlcCghTkUUzK.jpg,804251,Alcarràs,ca,Alcarràs,"For as long as they can remember, the Solé fam...",/93eceHkP3bTTGhrERgL6nKdYgNG.jpg,movie,[18],17.275,2022-04-29,False,7.0,68
2,tt10062338,Earwig (2021),adult backdrop_path ...,False,/qHAEM9QiplQEGhRa0cx3dmhIwE7.jpg,657826,Earwig,en,Earwig,"Somewhere in Europe, mid-20th century. Albert ...",/kroBXkrqoNG4x6unizzOEJ6kXkF.jpg,movie,"[18, 14, 27]",5.397,2022-06-15,False,6.2,24
3,tt14967958,Goodbye (2021),adult backdrop_path ...,False,/1Vt65frTrAutUeymz5KPYz4xozZ.jpg,846993,"Goodbye, Don Glees!",ja,グッバイ、ドン・グリーズ！,"Teen misfits Roma, Toto, and Drop call themsel...",/42bRH1aEUge6Iia7yS18XV3Wpjg.jpg,movie,"[16, 12, 14]",119.262,2022-02-18,False,4.8,5
4,tt14817802,Le Secret des Perlims (2022),adult backdrop_path ...,False,/cLvqmozY9pJX46rVDQWjhUndGed.jpg,829424,Perlimps,pt,Perlimps,"Claé and Bruô, secret agents from enemy Kingdo...",/fhb1IwiU2vO1MXMYRRIjF5NPAXt.jpg,movie,[16],9.856,2023-01-18,False,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,tt15239678,Dune: Deuxième partie (2023),adult backdrop_path ...,False,/iKOs7caYOHzsRNWfY9RfIpbuLeI.jpg,693134,Dune: Part Two,en,Dune: Part Two,Follow the mythic journey of Paul Atreides as ...,/fU7mxkzsAXioPvDT7b9ge79bnPj.jpg,movie,"[28, 12, 878]",14.542,2023-11-03,False,0.0,0
169,tt18224752,Complètement Cramé (2023),adult backdrop_path ...,False,/2O9X3K9h9COlQ1LHHEhBXHlSrUT.jpg,792614,Well Done,fr,Complètement Cramé !,,/hIQ11GZBYU4THpejvABJHWgjYkK.jpg,movie,"[18, 35, 10749]",0.754,2023-11-01,False,0.0,0
170,tt24243464,Robot Chicken (2023),Empty DataFrame Columns: [] Index: [],,,,,,,,,,,,,,,
171,tt12672620,Les trois mousquetaires: Milady (2023),adult backdrop_path id ...,False,None,845111,The Three Musketeers: Milady,fr,Les Trois Mousquetaires : Milady,Second half of a two-film adaptation of Alexan...,/5iODvV2VeTAGfW679PvCDQeqO98.jpg,movie,[12],4.742,2023-12-13,False,0.0,0


In [279]:
cast_json = []
crew_json = []
runtime = []
budget = []
for tmdb_id in liste_imdb.id_ :
    cast_json_temp = []
    crew_json_temp = []
    if str(tmdb_id) == '':
        cast_json.append('-')
        crew_json.append('-')
        runtime.append('-')
        budget.append('-')
    else:
        url_tmdb_cast_credit = 'https://api.themoviedb.org/3/movie/'+str(tmdb_id)+'/credits?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=fr-FR'
        response_cast_credit = requests.get(url_tmdb_cast_credit)
        json_data_cast_credit = response_cast_credit.json()
        cast_json.append(json_data_cast_credit['cast'])
        crew_json.append(json_data_cast_credit['crew'])
        url_tmdb_runtime_budget = 'https://api.themoviedb.org/3/movie/'+str(tmdb_id)+'?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=fr-FR'
        response_runtime_budget = requests.get(url_tmdb_runtime_budget)
        json_data_runtime_budget = response_runtime_budget.json()
        runtime.append(json_data_runtime_budget['runtime'])
        budget.append(json_data_runtime_budget['budget'])
        
liste_imdb['cast_json'] = cast_json
liste_imdb['crew_json'] = crew_json

liste_imdb['runtime'] = runtime
liste_imdb['budget'] = budget

In [280]:
liste_imdb.columns

Index(['id', 'titre', 'info', 'adult', 'backdrop_path', 'id_', 'title',
       'original_language', 'original_title', 'overview', 'poster_path',
       'media_type', 'genre_ids', 'popularity', 'release_date', 'video',
       'vote_average', 'vote_count', 'cast_json', 'crew_json', 'runtime',
       'budget'],
      dtype='object')

In [281]:
#Je recherche le nom correspondant au genre id 
#Je récupère le genre dans un dataframe
url_tmdb_genre = 'https://api.themoviedb.org/3/genre/movie/list?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=fr-FR'
response_genre = requests.get(url_tmdb_genre)
json_data_genre = response_genre.json()
df_genre = pd.DataFrame(json_data_genre['genres'])

genre_nom = []
for genre_list in liste_imdb.genre_ids : 
    genre_temp = []
    for g in genre_list : 
        for idg, n in zip(df_genre.id, df_genre.name):
            if g == idg :
                genre_temp.append(n)
    genre_nom.append(genre_temp)
    
liste_imdb.insert(13,'genre_nom',genre_nom)


In [284]:
#J'ajoute la date de naissaince et l'âge des membres du casting
birthdays = []
for cl in liste_imdb.cast_json :
    birthday_temp = []
    if cl != '-' :
        for c in cl :
            c_id = str(c['id'])
            url_cast_birthday = 'https://api.themoviedb.org/3/person/'+c_id+'?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=fr-FR'
            response_cast_birthday = requests.get(url_cast_birthday)
            json_data_cast_birthday = response_cast_birthday.json()
            if json_data_cast_birthday['birthday'] != None :
                age_calc = datetime.now().year-datetime.strptime(json_data_cast_birthday['birthday'], '%Y-%m-%d').date().year
                birthday_temp.append({'name' : c['name'], 'id' : c['id'], 'birthday' : json_data_cast_birthday['birthday'], 'age' : age_calc})
            else:
                birthday_temp.append({'name' : c['name'], 'id' : c['id'], 'birthday' : '-', 'age' : '-'})
        birthdays.append(birthday_temp)
    else :
        birthdays.append(('-'))
liste_imdb.insert(20,'cast_birthdays',birthdays)

In [291]:
liste_imdb.cast_birthdays[171]

[{'name': 'Eva Green', 'id': 10912, 'birthday': '1980-07-06', 'age': 43},
 {'name': 'Vincent Cassel', 'id': 1925, 'birthday': '1966-11-23', 'age': 57},
 {'name': 'Louis Garrel', 'id': 16269, 'birthday': '1983-06-14', 'age': 40},
 {'name': 'Vicky Krieps', 'id': 1080195, 'birthday': '1983-10-04', 'age': 40},
 {'name': 'Romain Duris', 'id': 17497, 'birthday': '1974-05-28', 'age': 49},
 {'name': 'François Civil', 'id': 78124, 'birthday': '1990-01-29', 'age': 33},
 {'name': 'Pio Marmaï', 'id': 146287, 'birthday': '1984-07-13', 'age': 39},
 {'name': 'Lyna Khoudri', 'id': 1886168, 'birthday': '1992-10-03', 'age': 31},
 {'name': 'Jacob Fortune-Lloyd', 'id': 1944459, 'birthday': '-', 'age': '-'},
 {'name': 'Eric Ruf', 'id': 229968, 'birthday': '1969-05-21', 'age': 54},
 {'name': 'Marc Barbé', 'id': 135665, 'birthday': '1961-05-06', 'age': 62},
 {'name': 'Patrick Mille', 'id': 56404, 'birthday': '1970-04-08', 'age': 53},
 {'name': 'Julien Frison', 'id': 35894, 'birthday': '1993-07-18', 'age': 30

In [292]:
#Export au format CSV
liste_imdb.to_csv('liste_film_imdb_20230115.csv',index=False, sep=';')

In [ ]:
api tmbd = 4f0d9b752dd1d3a0b158afb56be09ac7

### TMDB x BECHDEL TEST

### Bechdel API

 | Name | Value | 
| ----------- | ----------- |
 | visible | Has this movie been approved (currently only approved movies are returned, so this value will always be 1). | 
 | date | The date this movie was added to the list | 
 | submitterid | The ID of the submitter. Since submitter information is currently not available through the API, is of no use. | 
 | rating | The actual score. Number from 0 to 3 (0 means no two women, 1 means no talking, 2 means talking about a man, 3 means it passes the test). | 
 | dubious | Whether the submitter considered the rating dubious. | 
 | imdbid | The IMDb id. | 
 | id | The bechdeltest.com unique id. | 
 | title | The title of the movie. Any weird characters are HTML encoded (so Brüno is returned as "Br&uuml;no"). | 
 | year | The year this movie was released (according to IMDb). | 


In [5]:
url_bechdel = 'https://bechdeltest.com/api/v1/getMoviesByTitle?'
response_bt = requests.get(url_bechdel) # a supprimer dès que fini
json_data_bt = response_bt.json()
    #les données de navigation se trouvent dans la colonne catalogs du fichier Json
df_bt = json_normalize(json_data_bt)
df_bt=df_bt[["id","title","year","imdbid","visible","rating","dubious","date","submitterid"]]
df_bt.to_csv("bechdel.csv",index=False)
#df_bt

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


In [6]:
#je mets les id IMDB au format utiliser dans TMDB
df_bt['imdbID_for_tmdb']  = ['tt'+str(imid) for imid in df_bt.imdbid]
df_bt

,id,title,year,imdbid,visible,rating,dubious,date,submitterid,imdbID_for_tmdb
0,10195,"355, The",2022,8356942,1,3,0,2022-01-13 14:47:49,19837,tt8356942
1,10568,A Fada do Lar,2022,10541410,1,3,0,2022-10-20 19:25:34,20532,tt10541410
2,10263,"Adam Project, The",2022,2463208,1,1,0,2022-03-19 22:25:13,19968,tt2463208
3,10436,All the old knives,2022,3706352,1,3,1,2022-08-08 13:58:02,20339,tt3706352
4,10308,Allra s&iacute;&eth;asta vei&eth;ifer&eth;in,2022,15387742,1,1,0,2022-05-02 10:14:43,20065,tt15387742
...,...,...,...,...,...,...,...,...,...,...
9797,9806,Le singe musicien,1878,12592084,1,0,0,2021-05-11 23:38:54,19151,tt12592084
9798,9603,Sallie Gardner at a Gallop,1878,2221420,1,0,0,2021-04-03 02:25:27,18882,tt2221420
9799,9804,La Rosace Magique,1877,14495706,1,0,0,2021-05-11 00:11:22,19145,tt14495706
9800,9602,Passage de Venus,1874,3155794,1,0,0,2021-04-02 20:58:09,18880,tt3155794


In [7]:
#J'ajoute les métadonnées nécessaires, cast, genre, etc...

#Créaton des colonnes de base depuis IMDB
imdb_id_film=[]
imdb_titre=[]

#Création de la colonne des infos de film depuis l'API TMDB à l'aide de l'ID IMDB 
imdb_info=[] #liste titres trouvés sur MovieDB par le biais de l'api


#création de colonnes depuis imdb_info
adult = []
backdrop_path = []
id_ = []
title = []
original_language = []
original_title = []
overview = []
poster_path = []
media_type = []
genre_ids = []
popularity = []
release_date = []
video = []
vote_average = []
vote_count = []

for imdb_id in df_bt['imdbID_for_tmdb'] : 
    url_mdb = 'https://api.themoviedb.org/3/find/'+imdb_id+'?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=fr-FR&external_source=imdb_id'
    response_l = requests.get(url_mdb)
    json_data_l = response_l.json()
    df_n=pd.json_normalize(json_data_l['movie_results'])
    imdb_info.append(df_n)
  
  #ajout des info dans le dataframe df_n
    if len(df_n)==1 :
        adult.append(df_n['adult'][0])
        backdrop_path.append(df_n['backdrop_path'][0])
        id_.append(df_n['id'][0])
        title.append(df_n['title'][0])
        original_language.append(df_n['original_language'][0])
        original_title.append(df_n['original_title'][0])
        overview.append(df_n['overview'][0])
        poster_path.append(df_n['poster_path'][0])
        media_type.append(df_n['media_type'][0])
        genre_ids.append(df_n['genre_ids'][0])
        popularity.append(df_n['popularity'][0])
        release_date.append(df_n['release_date'][0])
        video.append(df_n['video'][0])
        vote_average.append(df_n['vote_average'][0])
        vote_count.append(df_n['vote_count'][0])
    else:
        adult.append('')
        backdrop_path.append('')
        id_.append('')
        title.append('')
        original_language.append('')
        original_title.append('')
        overview.append('')
        poster_path.append('')
        media_type.append('')
        genre_ids.append('')
        popularity.append('')
        release_date.append('')
        video.append('')
        vote_average.append('')
        vote_count.append('')
imdb_info

[   adult                     backdrop_path      id title original_language  \
 0  False  /8pgKccb5PfE1kWB9qqiXJem83VC.jpg  522016   355                en   
 
   original_title                                           overview  \
 0        The 355  Une arme technologique capable de prendre le c...   
 
                         poster_path media_type genre_ids  popularity  \
 0  /xOjRcxtCFup6UKtyDzfekwZOkBx.jpg      movie  [28, 53]      70.836   
 
   release_date  video  vote_average  vote_count  
 0   2022-01-05  False          6.16         749  ,
    adult backdrop_path       id          title original_language  \
 0  False          None  1033762  A Fada do Lar                pt   
 
   original_title overview                       poster_path media_type  \
 0  A Fada do Lar           /z9IwjN2Z4oQR5CpOq9nrVji83Q7.jpg      movie   
 
   genre_ids  popularity release_date  video  vote_average  vote_count  
 0      [35]        1.16   2022-10-05  False           5.0           3  ,
    

In [8]:
#Ajout de colonnes au dataframe à partir des listes créées
liste_col=['adult', 'backdrop_path', 'id_', 'title', 'original_language', 'original_title', 'overview', 'poster_path', 'media_type', 'genre_ids', 'popularity', 'release_date', 'video', 'vote_average', 'vote_count']
liste_liste = [adult, backdrop_path, id_, title, original_language, original_title, overview, poster_path, media_type, genre_ids, popularity, release_date, video, vote_average, vote_count]

for col, liste in zip(liste_col, liste_liste) :
    df_bt[col]=liste

In [11]:
#création des listes des membres du casting et de l'équipe tech
cast_json = []
crew_json = []
runtime = []
budget = []

for tmdb_id in df_bt.id_ :
    cast_json_temp = []
    crew_json_temp = []
    if str(tmdb_id) == '':
        cast_json.append('-')
        crew_json.append('-')
        runtime.append('-')
        budget.append('-')
    else:
        url_tmdb_cast_credit = 'https://api.themoviedb.org/3/movie/'+str(tmdb_id)+'/credits?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=fr-FR'
        response_cast_credit = requests.get(url_tmdb_cast_credit)
        json_data_cast_credit = response_cast_credit.json()
        cast_json.append(json_data_cast_credit['cast'])
        crew_json.append(json_data_cast_credit['crew'])
        
        url_tmdb_runtime_budget = 'https://api.themoviedb.org/3/movie/'+str(tmdb_id)+'?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=fr-FR'
        response_runtime_budget = requests.get(url_tmdb_runtime_budget)
        json_data_runtime_budget = response_runtime_budget.json()
        runtime.append(json_data_runtime_budget['runtime'])
        budget.append(json_data_runtime_budget['budget'])
        
df_bt['cast_json'] = cast_json
df_bt['crew_json'] = crew_json

df_bt['runtime'] = runtime
df_bt['budget'] = budget


In [14]:
#Je recherche le nom correspondant au genre id 
#Je récupère le genre dans un dataframe
url_tmdb_genre = 'https://api.themoviedb.org/3/genre/movie/list?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=fr-FR'
response_genre = requests.get(url_tmdb_genre)
json_data_genre = response_genre.json()
df_genre = pd.DataFrame(json_data_genre['genres'])

genre_nom = []
for genre_list in df_bt.genre_ids : 
    genre_temp = []
    for g in genre_list : 
        for idg, n in zip(df_genre.id, df_genre.name):
            if g == idg :
                genre_temp.append(n)
    genre_nom.append(genre_temp)
    
df_bt.insert(19,'genre_nom',genre_nom)

In [ ]:
#J'ajoute la date de naissaince et l'âge des membres du casting
birthdays = []
for cl in df_bt.cast_json :
    birthday_temp = []
    if cl != '-' :
        for c in cl :
            c_id = str(c['id'])
            url_cast_birthday = 'https://api.themoviedb.org/3/person/'+c_id+'?api_key=4f0d9b752dd1d3a0b158afb56be09ac7&language=fr-FR'
            response_cast_birthday = requests.get(url_cast_birthday)
            json_data_cast_birthday = response_cast_birthday.json()
            if json_data_cast_birthday['birthday'] != None :
                age_calc = datetime.now().year-datetime.strptime(json_data_cast_birthday['birthday'], '%Y-%m-%d').date().year
                birthday_temp.append({'name' : c['name'], 'id' : c['id'], 'birthday' : json_data_cast_birthday['birthday'], 'age' : age_calc})
            else:
                birthday_temp.append({'name' : c['name'], 'id' : c['id'], 'birthday' : '-', 'age' : '-'})
        birthdays.append(birthday_temp)
    else :
        birthdays.append(('-'))
df_bt.insert(26,'cast_birthdays',birthdays)

In [21]:
df_bt

,id,title,year,imdbid,visible,rating,dubious,date,submitterid,imdbID_for_tmdb,...,genre_nom,popularity,release_date,video,vote_average,vote_count,cast_json,crew_json,runtime,budget
0,10195,355,2022,8356942,1,3,0,2022-01-13 14:47:49,19837,tt8356942,...,"[Action, Thriller]",70.836,2022-01-05,False,6.16,749,"[{'adult': False, 'gender': 1, 'id': 83002, 'k...","[{'adult': False, 'gender': 2, 'id': 966, 'kno...",124,75000000
1,10568,A Fada do Lar,2022,10541410,1,3,0,2022-10-20 19:25:34,20532,tt10541410,...,[Comédie],1.16,2022-10-05,False,5.0,3,"[{'adult': False, 'gender': 1, 'id': 1150466, ...","[{'adult': False, 'gender': 0, 'id': 1944726, ...",103,0
2,10263,Adam à travers le temps,2022,2463208,1,1,0,2022-03-19 22:25:13,19968,tt2463208,...,"[Aventure, Science-Fiction]",112.748,2022-03-11,False,7.073,3315,"[{'adult': False, 'gender': 2, 'id': 10859, 'k...","[{'adult': False, 'gender': 2, 'id': 10956, 'k...",106,116000000
3,10436,Le Couteau par la lame,2022,3706352,1,3,1,2022-08-08 13:58:02,20339,tt3706352,...,"[Thriller, Action]",50.359,2022-04-08,False,6.092,397,"[{'adult': False, 'gender': 2, 'id': 62064, 'k...","[{'adult': False, 'gender': 1, 'id': 474, 'kno...",105,0
4,10308,Allra síðasta veiðiferðin,2022,15387742,1,1,0,2022-05-02 10:14:43,20065,tt15387742,...,[Comédie],0.932,2022-03-18,False,6.0,1,"[{'adult': False, 'gender': 2, 'id': 52913, 'k...","[{'adult': False, 'gender': 2, 'id': 79815, 'k...",95,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9797,9806,Le Singe Musicien,1878,12592084,1,0,0,2021-05-11 23:38:54,19151,tt12592084,...,[Animation],0.716,1878-05-07,False,6.458,12,[],"[{'adult': False, 'gender': 2, 'id': 935707, '...",1,153
9798,9603,Sallie Gardner au galop,1878,2221420,1,0,0,2021-04-03 02:25:27,18882,tt2221420,...,"[Animation, Documentaire]",5.524,1878-06-14,False,6.793,128,"[{'adult': False, 'gender': 1, 'id': 1727031, ...","[{'adult': False, 'gender': 2, 'id': 1077300, ...",1,0
9799,9804,La Rosace Magique,1877,14495706,1,0,0,2021-05-11 00:11:22,19145,tt14495706,...,[Animation],0.6,1878-05-07,False,6.333,9,[],"[{'adult': False, 'gender': 2, 'id': 935707, '...",1,0
9800,9602,Passage de Venus,1874,3155794,1,0,0,2021-04-02 20:58:09,18880,tt3155794,...,[Documentaire],4.606,1874-12-09,False,6.197,76,[],"[{'adult': False, 'gender': 2, 'id': 1456058, ...",1,0


In [20]:
df_bt.to_csv('liste_film_imdb_20230116.csv',index=False, sep=';')

In [ ]:
#Export au format CSV
df_bt.to_csv('liste_film_imdb_20230115.csv',index=False, sep=';')

### films from cinefil

In [199]:
url_senscritique = 'https://www.cinefil.com/films-a-venir/annee-2023?page=1'
senscritique = requests.get(url_senscritique).content
senscritique
senscritique_soup = BeautifulSoup(senscritique,"lxml")
senscritique_soup
data_film_sc=senscritique_soup.findAll('div',attrs={"class":"fiche-infos2"})
data_film_sc

[]

In [201]:
#Simple assignment
from selenium.webdriver import Firefox

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#nb de pages 1 à 132
nombre=range(1,10)

driver = Firefox()
liste_lib=[]
nom_lib = [] 
description = []
adresse = []
tags_clickcollect =[]
tags_livraison =[]
tags_aemporter =[]

for t in nombre:
    #try:
    time.sleep(3)
    url=f'https://www.cinefil.com/films-a-venir/annee-2023?page={t}'
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    data=soup.findAll('div',attrs={"class":"wemap-pinpoint-cell-text col-center"})
    liste_lib.append(data)

NoSuchWindowException: Message: Browsing context has been discarded


### Etape 3
Après plusieurs tentatives, J'ai constaté qu'il n'était pas possible de scraper la partie droite du site. D'une part site le chargement se faisait sous la forme d'un "embed" d'autre part parce que les infos à scraper se trouve, en analysant le code, dans un bloc caché (on le déduit avec le CSS "overflow: hidden;").
Après quelques recherches, et une analyse plus approfondi du code, qu'il fallait dans un premier temps passer au format "iframe". De plus, il me fallait lancer les pages à l'aide de sélénuim et en enfin faire le scraping avec BS.
### Etape 4
Pour construire le tableau, j'ai identifié les bouts de code qui me seraient nécessaires. Je les ai 

In [5]:
#!/usr/bin/env python

#Simple assignment
from selenium.webdriver import Firefox

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#nb de pages 1 à 132
nombre=range(1,132)

driver = Firefox()
liste_lib=[]
nom_lib = [] 
description = []
adresse = []
tags_clickcollect =[]
tags_livraison =[]
tags_aemporter =[]

for t in nombre:
    #try:
    time.sleep(3)
    url=f'https://livemap.getwemap.com/iframe.html?emmid=14887&token=47DPO87WKCOPFJ1OH9QZ9NSL2&wemapHash=%2523%252Fsearch%254046.5761946%252C2.2467858%252C5.43&iframeid=18dcff69-bcfa-4c85-a9a4-91ae82f89419&parentOrigin=https://livemap.getwemap.com#/search?page={t}@46.5761946,2.2467858,5.43'
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    data=soup.findAll('div',attrs={"class":"wemap-pinpoint-cell-text col-center"})
    for d in data : 
        nom_lib.append([x.text for x in d.findAll('h3',attrs={"class":"cell-title"})])
        description.append([x.text for x in d.findAll('div',attrs={"class":"cell-description"})])
        adresse.append([x.text for x in d.findAll('span',attrs={"class":"cell-address"})])
        tags_clickcollect.append([x.text for x in d.findAll('div',attrs={"class":"click-collect"})])
        tags_livraison.append([x.text for x in d.findAll('div',attrs={"class":"livraison"})])
        tags_aemporter.append([x.text for x in d.findAll('div',attrs={"class":"à-emporter"})])
        
    liste_lib.append(data)
    #except:
       # driver.quit()

In [6]:
liste_lib

[[],
 [<div class="wemap-pinpoint-cell-text col-center" data-radium="true"><div><h3 class="cell-title"><b>Le Grenier Fort</b></h3></div><div class="cell-description"> Téléphone
  0384416729
   Email
  le.grenier.fort@orange.fr
  </div><span class="cell-address">5 rue de Genève  39150 Saint Laurent en Grandvaux</span><div class="tags-box"><div class="sc-fzXfQq cTmeZn tag disabled click-collect noicon"><span class="sc-fzXfQr cFAcpo">Click and collect</span></div></div></div>,
  <div class="wemap-pinpoint-cell-text col-center" data-radium="true"><div><h3 class="cell-title"><b>anagramme</b></h3></div><div class="cell-description"> Type de librairie
  Généraliste
   Téléphone
  0603476130
   Email
  romaneserda@hotmail.com
  </div><span class="cell-address">160 rue de Paris  92190 Meudon</span><div class="tags-box"><div class="sc-fzXfQq cTmeZn tag disabled click-collect noicon"><span class="sc-fzXfQr cFAcpo">Click and collect</span></div></div></div>,
  <div class="wemap-pinpoint-cell-text 

In [7]:
dflib = pd.DataFrame(list(zip(nom_lib,description,adresse,tags_clickcollect,tags_livraison,tags_aemporter)),columns=['nom_lib','description','adresse','tags_clickcollect','tags_livraison','tags_aemporter'])

In [8]:
dflib

,nom_lib,description,adresse,tags_clickcollect,tags_livraison,tags_aemporter
0,[Le Grenier Fort],[ Téléphone\n0384416729\n Email\nle.grenier.fo...,[5 rue de Genève 39150 Saint Laurent en Grand...,[Click and collect],[],[]
1,[anagramme],[ Type de librairie\nGénéraliste\n Téléphone\n...,[160 rue de Paris 92190 Meudon],[Click and collect],[],[]
2,[Librairie ACACIA],"[Bonjour, ma librairie existe depuis 2003, je ...",[49 rue MONTESQUIEU 33500 LIBOURNE],[Click and collect],[Livraison],[À emporter]
3,[Librairie de la Trinité],[ Type de librairie\nGénéraliste\n Téléphone\n...,[19 rue de la trinité 31000 Toulouse],[Click and collect],[Livraison],[À emporter]
4,[L'Encre bleue],[Retrait des commandes du lundi au vendredi de...,[16 rue Saint Sauveur 50400 Granville],[Click and collect],[Livraison],[À emporter]
...,...,...,...,...,...,...
1295,"[La Marge, Ajaccio]",[],"[4 Rue Emmanuel Arène, 20000 Ajaccio, France]",[Click and collect],[],[]
1296,"[A Piuma Lesta, Bastia]",[],"[Centre Commercial Le Polygone, 20600 Bastia, ...",[Click and collect],[],[]
1297,"[Les Schistes Bleus, Cherbourg-en-Contentin]",[],"[10 Rue François La Vieille, 50100 Cherbourg-e...",[Click and collect],[],[]
1298,"[Les Temps Modernes, Orléans]",[],"[57 Rue Notre Dame de Recouvrance, 45000 Orléa...",[Click and collect],[],[]


In [9]:
Librairies=[]
for tag in dflib['nom_lib']:
    if len(tag)>0: 
        for t in tag :
            Librairies.append(t)
    else:
        Librairies.append('')

Description=[]
for tag in dflib['description']:
    if len(tag)>0: 
        for t in tag :
            Description.append(t)
    else:
        Description.append('')
Adresse=[]
for tag in dflib['adresse']:
    if len(tag)>0: 
        for t in tag :
            Adresse.append(t)
    else:
        Adresse.append('')
        
clickcollect=[]
for tag in dflib['tags_clickcollect']:
    if len(tag)>0: 
        for t in tag :
            clickcollect.append(t)
    else:
        clickcollect.append('')

livraison=[]
for tag in dflib['tags_livraison']:
    if len(tag)>0: 
        for t in tag :
            livraison.append(t)
    else:
        livraison.append('')
        
aemporter=[]
for tag in dflib['tags_aemporter']:
    if len(tag)>0: 
        for t in tag :
            aemporter.append(t)
    else:
        aemporter.append('')  

In [10]:
dflib['Librairies']=Librairies
dflib['Description']=Description
dflib['Adresse']=Adresse
dflib['click_collect']=clickcollect
dflib['livraison']=livraison
dflib['a_emporter']=aemporter

In [11]:
dflib

,nom_lib,description,adresse,tags_clickcollect,tags_livraison,tags_aemporter,Librairies,Description,Adresse,click_collect,livraison,a_emporter
0,[Le Grenier Fort],[ Téléphone\n0384416729\n Email\nle.grenier.fo...,[5 rue de Genève 39150 Saint Laurent en Grand...,[Click and collect],[],[],Le Grenier Fort,Téléphone\n0384416729\n Email\nle.grenier.for...,5 rue de Genève 39150 Saint Laurent en Grandvaux,Click and collect,,
1,[anagramme],[ Type de librairie\nGénéraliste\n Téléphone\n...,[160 rue de Paris 92190 Meudon],[Click and collect],[],[],anagramme,Type de librairie\nGénéraliste\n Téléphone\n0...,160 rue de Paris 92190 Meudon,Click and collect,,
2,[Librairie ACACIA],"[Bonjour, ma librairie existe depuis 2003, je ...",[49 rue MONTESQUIEU 33500 LIBOURNE],[Click and collect],[Livraison],[À emporter],Librairie ACACIA,"Bonjour, ma librairie existe depuis 2003, je l...",49 rue MONTESQUIEU 33500 LIBOURNE,Click and collect,Livraison,À emporter
3,[Librairie de la Trinité],[ Type de librairie\nGénéraliste\n Téléphone\n...,[19 rue de la trinité 31000 Toulouse],[Click and collect],[Livraison],[À emporter],Librairie de la Trinité,Type de librairie\nGénéraliste\n Téléphone\n0...,19 rue de la trinité 31000 Toulouse,Click and collect,Livraison,À emporter
4,[L'Encre bleue],[Retrait des commandes du lundi au vendredi de...,[16 rue Saint Sauveur 50400 Granville],[Click and collect],[Livraison],[À emporter],L'Encre bleue,Retrait des commandes du lundi au vendredi de ...,16 rue Saint Sauveur 50400 Granville,Click and collect,Livraison,À emporter
...,...,...,...,...,...,...,...,...,...,...,...,...
1295,"[La Marge, Ajaccio]",[],"[4 Rue Emmanuel Arène, 20000 Ajaccio, France]",[Click and collect],[],[],"La Marge, Ajaccio",,"4 Rue Emmanuel Arène, 20000 Ajaccio, France",Click and collect,,
1296,"[A Piuma Lesta, Bastia]",[],"[Centre Commercial Le Polygone, 20600 Bastia, ...",[Click and collect],[],[],"A Piuma Lesta, Bastia",,"Centre Commercial Le Polygone, 20600 Bastia, F...",Click and collect,,
1297,"[Les Schistes Bleus, Cherbourg-en-Contentin]",[],"[10 Rue François La Vieille, 50100 Cherbourg-e...",[Click and collect],[],[],"Les Schistes Bleus, Cherbourg-en-Contentin",,"10 Rue François La Vieille, 50100 Cherbourg-en...",Click and collect,,
1298,"[Les Temps Modernes, Orléans]",[],"[57 Rue Notre Dame de Recouvrance, 45000 Orléa...",[Click and collect],[],[],"Les Temps Modernes, Orléans",,"57 Rue Notre Dame de Recouvrance, 45000 Orléan...",Click and collect,,


In [12]:
dflib.drop(['nom_lib','description','adresse','tags_clickcollect','tags_livraison','tags_aemporter'], axis=1,inplace = True) 

In [13]:
dflib

,Librairies,Description,Adresse,click_collect,livraison,a_emporter
0,Le Grenier Fort,Téléphone\n0384416729\n Email\nle.grenier.for...,5 rue de Genève 39150 Saint Laurent en Grandvaux,Click and collect,,
1,anagramme,Type de librairie\nGénéraliste\n Téléphone\n0...,160 rue de Paris 92190 Meudon,Click and collect,,
2,Librairie ACACIA,"Bonjour, ma librairie existe depuis 2003, je l...",49 rue MONTESQUIEU 33500 LIBOURNE,Click and collect,Livraison,À emporter
3,Librairie de la Trinité,Type de librairie\nGénéraliste\n Téléphone\n0...,19 rue de la trinité 31000 Toulouse,Click and collect,Livraison,À emporter
4,L'Encre bleue,Retrait des commandes du lundi au vendredi de ...,16 rue Saint Sauveur 50400 Granville,Click and collect,Livraison,À emporter
...,...,...,...,...,...,...
1295,"La Marge, Ajaccio",,"4 Rue Emmanuel Arène, 20000 Ajaccio, France",Click and collect,,
1296,"A Piuma Lesta, Bastia",,"Centre Commercial Le Polygone, 20600 Bastia, F...",Click and collect,,
1297,"Les Schistes Bleus, Cherbourg-en-Contentin",,"10 Rue François La Vieille, 50100 Cherbourg-en...",Click and collect,,
1298,"Les Temps Modernes, Orléans",,"57 Rue Notre Dame de Recouvrance, 45000 Orléan...",Click and collect,,


In [14]:
csv_data = dflib.to_csv('20201113_data.csv',index=False, sep=';')